# Full-Text Search (FTS) in SurrealEngine

SurrealEngine uses SurrealDB's robust Full-Text Search features via the `@@` operator. Setting up FTS on your models involves adding parameters to `StringField`s, or defining them comprehensively in the `Meta` inner class for your Document.

In [ ]:
import asyncio
from surrealengine import (
    Document, StringField, 
    create_connection
)
from surrealengine.logging import logger
logger.set_level(30) # WARNING

db = create_connection(
    url="memory://",
    namespace="test_ns",
    database="test_fts",
    make_default=True,
    async_mode=True
)
await db.connect()

## Setting up the Schema

To define FTS indexes, you can flag it on the `StringField` directly, or you can register an index passing `"search": True` in `Meta.indexes`. Note that the analyzer you reference *must* exist within SurrealDB natively.

Here we'll manually specify an english analyzer before schema generation via raw query.

In [ ]:
await db.client.query("DEFINE ANALYZER english TOKENIZERS blank,class,camel,punct FILTERS snowball(english);")

class Article(Document):
    title = StringField(required=True)
    # Method 1: Field-level Search Registration
    summary = StringField(search=True, analyzer="english", bm25=True, indexed=True)
    content = StringField()
    
    class Meta:
        indexes = [
            # Method 2: Meta Search Registration
            {
                "name": "idx_content",
                "fields": ["content"],
                "search": True,
                "bm25": True,
                "highlights": True,
                "analyzer": "english"
            }
        ]

await Article.create_table(schemafull=True)

## Populating Search Content

Now we'll dump some posts into the database.

In [ ]:
a1 = Article(
    title="Python in 2026", 
    summary="A look back at python's journey into the future.", 
    content="Python has become the premier operating language for agents!"
)
a2 = Article(
    title="SurrealDB Rocks", 
    summary="A multi-model database for modern computing.", 
    content="We love using SurrealDB alongside Python and graph theory."
)
a3 = Article(
    title="Object-Relational Mapping basics",
    summary="Introduction to databases.",
    content="Developers use ORMs to write abstractions over SQL queries safely."
)
await a1.save()
await a2.save()
await a3.save()

## querying FTS using `QuerySet.search()`

The `.search(text, *fields)` method compiles into `SELECT * FROM article WHERE fields @@ 'text'` natively mapped to the specified SurrealDB FTS Index! We can pass raw column names as strings or we can provide the pythonic variables.

In [ ]:
res1 = await Article.objects.search("python", Article.summary).all()
print("Search 'python' in SUMMARY:")
for a in res1:
    print(" - ", a.title)

In [ ]:
res2 = await Article.objects.search("graph theory", "content").all()
print("Search 'graph theory' in CONTENT:")
for a in res2:
    print(" - ", a.title)

## Hybrid Multi-field Search

You can combine inputs on `.search()` to OR across multiple fields.

In [ ]:
res3 = await Article.objects.search("databases", Article.summary, Article.content).all()
print("Search 'databases' across SUMMARY and CONTENT:")
for a in res3:
    print(" - ", a.title)

await db.disconnect()